In [ ]:
import numpy as np
from lstchain.image.muon.muon_analysis import pixel_coords_to_telescope
from ctapipe.io import EventSource
import matplotlib.pyplot as plt
from lstchain.reco import r0_to_dl1
from ctapipe.image import ImageProcessor
from ctapipe.image.muon import MuonProcessor
from ctapipe.calib import CameraCalibrator
import pandas as pd
from astropy.table import Table, vstack

import glob

filename = '/Users/vdk//muons2024/data/for_comparison/nsb_tuning/triple_tuning/run101_muon.simtel.gz'
source = EventSource(filename, max_events = 5000)

In [ ]:
tel_description = source.subarray.tels[1]
geom = tel_description.camera.geometry
equivalent_focal_length = tel_description.optics.equivalent_focal_length
x_coord, y_coord = pixel_coords_to_telescope(geom, equivalent_focal_length)

### How to sum up the binned intensity

In [ ]:
alpha_angles = np.array([0,1,0,2,0])  # Your list of bin indices (1855 elements, ranging from 0 to 359)
image_clean = np.array([5,15,5,15,5])  # Your list of intensity values (1855 elements)

# Number of bins
num_bins = 5

# Initialize an array to hold the summed values for each bin
binned_sums = np.zeros(num_bins)

# Use np.add.at to accumulate values into the corresponding bins
np.add.at(binned_sums, alpha_angles, image_clean)
binned_sums

In [ ]:
def find_alpha_angles(x, y, x_center, y_center):
    """
    It should return the array with 1855 elements, each element contain angle with respect to the muon ring
    """
    # Calculate the angle with respect to the muon ring
    alpha_angles = np.rad2deg(np.arctan2(y - y_center, x - x_center))
    alpha_angles = np.round(alpha_angles).astype(int)
    alpha_angles[alpha_angles < 0] += 360
    return np.array(alpha_angles)

def intensity_binning(x, y, x_center, y_center, image, clean_mask):
    """
    It should return the array with 360 elements, each element contain number of p.e. in the bin width 1 deg
    """
    # Its a list with angles, where to each x,y element of pixel we map the angle with respect to this particular muon ring. Each element should be rounded to integer, to serve as a bin.
    
    alpha_angles = find_alpha_angles(x, y, x_center, y_center)
    # print(f"Alpha angles shape : {alpha_angles.shape}")
    # print(f"Alpha angles : {alpha_angles}")
    intensity_azimuth = np.zeros(360)
    # print(f"Intensity azimuth shape : {intensity_azimuth.shape}")
    # print(f"Image clean shape : {image.shape}")
    print(np.max(alpha_angles))  # Check if this exceeds 359
    print(np.min(alpha_angles))  # Should be >= 0
    print(np.shape(intensity_azimuth), np.shape(alpha_angles), np.shape(image))
    print(intensity_azimuth[359])
    image[~clean_mask] = 0
    np.add.at(intensity_azimuth, alpha_angles, image)
    return intensity_azimuth


## Tests

In [ ]:
filename = '/Users/vdk//muons2024/data/for_comparison/nsb_tuning/triple_tuning/run101_muon.simtel.gz'
source = EventSource(filename, max_events = 50)
event_iterator = iter(source)
events = np.array([])
for event in event_iterator:
        events = np.append(events, event)

In [ ]:
event = events[0]
image_processor = ImageProcessor(source.subarray)
muon_processor = MuonProcessor(source.subarray)
calib = CameraCalibrator(image_extractor_type="GlobalPeakWindowSum",subarray = source.subarray)

calib(event)
image_processor(event)
muon_processor(event)

In [ ]:
min(y_coord.degree)

In [ ]:
test_alpha = find_alpha_angles(x_coord.degree, y_coord.degree, 0.3,0)
test_alpha

In [ ]:
test_alpha = find_alpha_angles(x_coord.degree, y_coord.degree, 0,0)
z,x,c = plt.hist(test_alpha, bins = 10)

In [ ]:
event.muon.tel[1].ring.center_fov_lat

In [ ]:
alpha_angles = find_alpha_angles(x_coord.degree, y_coord.degree, 0, 0)
image = event.dl1.tel[1].image

# Ensure the shapes match
if alpha_angles.shape != image.shape:
    raise ValueError(f"Shape mismatch: alpha_angles shape {alpha_angles.shape} and image shape {image.shape}")

test_intensity = intensity_binning(x_coord.degree, y_coord.degree, 0.0639, 0.39, image, event.dl1.tel[1].image_mask)


In [ ]:
x_values = [i for i in range(360)]
#plt.scatter(x_values,test_intensity)
#plt.hist(test_intensity, bins = 10)
plt.plot(test_intensity)

In [ ]:
test_intensity

In [ ]:
image = event.dl1.tel[1].image
image_mask = event.dl1.tel[1].image_mask

image[~image_mask] = 0

In [ ]:
image

In [ ]:
print(event.dl1.tel[1].image[event.dl1.tel[1].image_mask])

In [ ]:
plt.plot(test_intensity)

In [ ]:
# Sample data: Replace these with your actual list1 and list2
list1 = alpha_angles
list2 = [10, 20, 5, 15, 25, 30]  # Values corresponding to each bin

# Number of bins (maximum bin number + 1, if bins start from 0)
num_bins = max(list1) + 1

# Initialize an array to store the sum of values for each bin
binned_values = np.zeros(num_bins)

# Aggregate values into bins
for bin_num, value in zip(list1, list2):
    binned_values[bin_num] += value

# Create the histogram (bar chart)
plt.bar(range(num_bins), binned_values)
plt.xlabel('Bin Number')
plt.ylabel('Summed Values')
plt.title('Histogram of Binned Values')
plt.xticks(range(num_bins))  # Ensure the bin numbers are shown correctly on x-axis
plt.show()

## Next step

### From ctapipe intensity_fitter.py

In [ ]:
# center_x: float
#     center of muon ring in the telescope frame
# pixel_x: quantity[angle]
#     Pixel x coordinate in telescope frame



dx = pixel_x_rad - center_x_rad
dy = pixel_y_rad - center_y_rad
ang = np.arctan2(dy, dx)

## Final version

In [ ]:
filename = '/Users/vdk//muons2024/data/for_comparison/nsb_tuning/triple_tuning/run101_muon.simtel.gz'
source = EventSource(filename, max_events = 50)
event_iterator = iter(source)
event = next(event_iterator)
image_processor = ImageProcessor(source.subarray)
muon_processor = MuonProcessor(source.subarray)
calib = CameraCalibrator(image_extractor_type="GlobalPeakWindowSum",subarray = source.subarray)
calib(event)
image_processor(event)
muon_processor(event)

tel_description = source.subarray.tels[1]
geom = tel_description.camera.geometry
equivalent_focal_length = tel_description.optics.equivalent_focal_length
x_coord, y_coord = pixel_coords_to_telescope(geom, equivalent_focal_length)

### Functions to extract azimuthal angles and bin the intensity to this angles

In [ ]:
def find_alpha_angles(x, y, x_center, y_center):
    """
    It should return the array with 1855 elements, each element contain angle with respect to the muon ring
    """
    # Calculate the angle with respect to the center of muon ring
    alpha_angles = np.rad2deg(np.arctan2(y - y_center, x - x_center))
    alpha_angles = np.round(alpha_angles).astype(int)
    alpha_angles[alpha_angles < 0] += 360
    return np.array(alpha_angles)

def intensity_binning(x, y, x_center, y_center, image, clean_mask):
    """
    It should return the array with 360 elements, each element contain number of p.e. in the bin width 1 deg with respect to muon center
    """
    # Its a list with angles, where to each x,y element of pixel 
    # we map the angle with respect to this particular muon ring. 
    # Each element is rounded to integer, to serve as a bin.
    alpha_angles = find_alpha_angles(x, y, x_center, y_center)
    intensity_azimuth = np.zeros(360)
    image[~clean_mask] = 0
    np.add.at(intensity_azimuth, alpha_angles, image)
    return intensity_azimuth


In [ ]:
alpha_angles = find_alpha_angles(x_coord.degree, y_coord.degree, 0, 0)
image = event.dl1.tel[1].image

# Ensure the shapes match
if alpha_angles.shape != image.shape:
    raise ValueError(f"Shape mismatch: alpha_angles shape {alpha_angles.shape} and image shape {image.shape}")

test_intensity = intensity_binning(x_coord.degree, y_coord.degree, 0.0639, 0.39, image, event.dl1.tel[1].image_mask)


In [ ]:
center_x = event.muon.tel[1].ring.center_fov_lon.to_value()
center_y = event.muon.tel[1].ring.center_fov_lat.to_value()
intensity_profile_temp = intensity_binning(x_coord.degree, y_coord.degree, center_x, center_y, event.dl1.tel[1].image, event.dl1.tel[1].image_mask)

In [ ]:
filename = '/Users/vdk//muons2024/data/for_comparison/nsb_tuning/triple_tuning/run101_muon.simtel.gz'
source = EventSource(filename, max_events = 100)
event_iterator = iter(source)
image_processor = ImageProcessor(source.subarray)
muon_processor = MuonProcessor(source.subarray)
calib = CameraCalibrator(image_extractor_type="GlobalPeakWindowSum",subarray = source.subarray)

intensity_profiles = []

for event in event_iterator:
        calib(event)
        image_processor(event)
        muon_processor(event)
        
        center_x = event.muon.tel[1].ring.center_fov_lon.to_value()
        center_y = event.muon.tel[1].ring.center_fov_lat.to_value()
        #if event.muon.tel[1].parameters.completeness > 0.99:
        if event.muon.tel[1].efficiency.impact.to_value() > 8:
                intensity_profile_temp = intensity_binning(x_coord.degree, y_coord.degree, center_x, center_y, event.dl1.tel[1].image, event.dl1.tel[1].image_mask)
        
                intensity_profiles.append(intensity_profile_temp)

intensity_profiles = np.array(intensity_profiles)
        

In [ ]:
for shape in intensity_profiles:
    #plt.scatter(x_values,shape)
    plt.plot(shape,alpha=1)

    #plt.ylim(0,100)

In [ ]:
event.muon.tel[1].parameters.completeness

In [ ]:
import matplotlib.pyplot as plt

# Assuming intensity_profiles is a list of 10 lists, each representing a profile
for i, shape in enumerate(intensity_profiles):
    plt.figure()  # Create a new figure for each profile
    plt.plot(shape, alpha=1)  # Plot the profile
    plt.title(f'Profile {i+1}')  # Add a title to each plot
    plt.xlabel('X Values')  # Label for the x-axis (optional)
    plt.ylabel('Y Values')  # Label for the y-axis (optional)
    plt.show()  # Show the plot

In [ ]:
event.muon.tel[1].efficiency.impact

## Revised version

In [ ]:
def find_alpha_angles(x, y, x_center, y_center, x_cam, y_cam):
    # Calculate the azimuthal angles relative to the ring center
    alpha_angles = np.rad2deg(np.arctan2(y - y_center, x - x_center))
    alpha_angles[alpha_angles < 0] += 360

    # Calculate the reference angle from the ring center to the camera center
    phi0 = np.rad2deg(np.arctan2(y_center - y_cam, x_center - x_cam))
    if phi0 < 0:
        phi0 += 360

    # Adjust angles to have a common zero pointing towards the camera center
    alpha_angles_adjusted = alpha_angles - phi0
    alpha_angles_adjusted[alpha_angles_adjusted < 0] += 360
    return alpha_angles_adjusted

def intensity_binning(x, y, x_center, y_center, image, clean_mask, bin_size=10, x_cam=0, y_cam = 0):
    alpha_angles = find_alpha_angles(x, y, x_center, y_center, x_cam, y_cam)
    num_bins = int(360 / bin_size)
    intensity_azimuth = np.zeros(num_bins)
    image[~clean_mask] = 0

    # Map adjusted angles to bins
    bin_indices = (alpha_angles // bin_size).astype(int) % num_bins
    np.add.at(intensity_azimuth, bin_indices, image)
    return intensity_azimuth

In [ ]:
filename = '/Users/vdk//muons2024/data/for_comparison/nsb_tuning/triple_tuning/run101_muon.simtel.gz'
source = EventSource(filename, max_events = 1000)
event_iterator = iter(source)
image_processor = ImageProcessor(source.subarray)
muon_processor = MuonProcessor(source.subarray)
calib = CameraCalibrator(image_extractor_type="GlobalPeakWindowSum",subarray = source.subarray)

intensity_profiles = []

for event in event_iterator:
        calib(event)
        image_processor(event)
        muon_processor(event)
        
        center_x = event.muon.tel[1].ring.center_fov_lon.to_value()
        center_y = event.muon.tel[1].ring.center_fov_lat.to_value()
        if event.muon.tel[1].parameters.completeness > 0.9:
        #if event.muon.tel[1].efficiency.impact.to_value() < 3:
                intensity_profile_temp = intensity_binning(
                        x_coord.degree, 
                        y_coord.degree, 
                        center_x, 
                        center_y, 
                        event.dl1.tel[1].image, 
                        event.dl1.tel[1].image_mask
                )
        
                intensity_profiles.append(intensity_profile_temp)
        if len(intensity_profiles) == 10:
            break

intensity_profiles = np.array(intensity_profiles)
        

In [ ]:
bin_size = 10  # You can adjust this to 5, 10, etc.
num_bins = int(360 / bin_size)
bin_edges = np.linspace(0, 360, num_bins + 1)
bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2
plt.figure(figsize=(10, 6))

for shape in intensity_profiles[1:2]:
    #plt.scatter(x_values,shape)
    #plt.plot(shape,alpha=1)    
    plt.step(bin_centers, shape, where='mid', linewidth=2)

    #plt.ylim(0,100)

In [ ]:
intensity_profiles.shape

### Data

In [ ]:
image_folder = glob.glob('/Users/vdk/muons2024/data/17043/dl1_image/*')
mask_folder = glob.glob('/Users/vdk/muons2024/data/17043/dl1_mask/*')
peak_folder = glob.glob('/Users/vdk/muons2024/data/17043/dl1_peak/*')

data_dl1_images = []
data_dl1_mask = []
data_dl1_peak = []

for file in image_folder:
    tmp_result = np.load(file)
    data_dl1_images.append(tmp_result)

for file in mask_folder:
    tmp_result = np.load(file)
    data_dl1_mask.append(tmp_result)

for file in peak_folder:
    tmp_result = np.load(file)
    data_dl1_peak.append(tmp_result)

In [ ]:
file_path = '/Users/vdk/muons2024/data/dl1_LST-1.Run17043.0001.h5'

# Open the .h5 file
with h5py.File(file_path, 'r') as f:
    # Access the dataset
    dataset = f['dl1/event/telescope/image/LST_LSTCam']
    print(f"dataset {dataset}")
    print(f"dataset type {type(dataset)}")
    # Read the data
    data = dataset[:]
    print(f"data {data}")
    print(f"data type {type(data)}")
    
    # Print some information about the data
    # print(f'Dataset shape: {dataset.shape}')
    # print(f'Dataset dtype: {dataset.dtype}')
    # print('Data:', data)

    # If you want to read a specific part of the data, you can use slicing
    # For example, to read the first 10 entries:
    partial_data = dataset[:10]
#    print('First 10 entries:', partial_data)

In [ ]:
data_loaded = pd.read_csv('/Users/vdk/Software/code/muon_paper_2024/low_nsb_2024year.csv', index_col=0)

In [ ]:
data_loaded[100:130]

In [ ]:
fits_file = '/Users/vdk/muons2024/data/additional_statistic/muons_LST-1.Run17743.fits'

dat = Table.read(fits_file, format='fits')
dat['good_ring'] = dat['good_ring'].astype(bool)
df = dat.to_pandas()
df_good_data = df[(df['muon_efficiency'] < 1) & 
                      (df['size_outside'] < 500)]

df_good_data

ring_completeness_cut = 0.9
ring_containment_cut = 0.99
event_ids = np.array(list(df_good_data['event_id'][(df_good_data['ring_completeness'] > ring_completeness_cut) &
                                           (df_good_data['ring_containment'] > ring_containment_cut)]))

min_impact = min(df_good_data['impact_parameter'])
max_impact = max(df_good_data['impact_parameter']) 

filename_event_ids = '/Users/vdk/muons2024/data/17043_muons_08complet.txt'
#np.save(filename_event_ids, event_ids)

df_real_data = df_good_data[(df_good_data['ring_completeness'] > ring_completeness_cut) &
                                           (df_good_data['ring_containment'] > ring_containment_cut)]

In [ ]:
plt.hist(df_real_data['ring_width'], bins = 40)

In [ ]:
run_number = 17769
fits_file = f'/Users/vdk/muons2024/data/additional_statistic/muons_LST-1.Run{run_number}.fits'

dat = Table.read(fits_file, format='fits')
dat['good_ring'] = dat['good_ring'].astype(bool)
df = dat.to_pandas()
df_good_data = df[(df['muon_efficiency'] < 1) & 
                      (df['size_outside'] < 500)]

df_good_data

ring_completeness_cut = 0.4
ring_containment_cut = 0.99
event_ids = np.array(list(df_good_data['event_id'][(df_good_data['ring_completeness'] > ring_completeness_cut) &
                                           (df_good_data['ring_containment'] > ring_containment_cut)]))

min_impact = min(df_good_data['impact_parameter'])
max_impact = max(df_good_data['impact_parameter']) 

filename_event_ids = f'/Users/vdk/muons2024/data/additional_statistic/event_ids/{run_number}_muons_complet04_contain099.txt'
np.save(filename_event_ids, event_ids)

df_real_data = df_good_data[(df_good_data['ring_completeness'] > ring_completeness_cut) &
                                           (df_good_data['ring_containment'] > ring_containment_cut)]

In [ ]:
df_real_data

In [ ]:
#!/usr/bin/env python3
import h5py
import hdf5plugin
import glob
import numpy as np
import argparse


parser = argparse.ArgumentParser(description='Process run_number and date.')
parser.add_argument('run_number', type=str, help='The run number.')
parser.add_argument('date', type=str, help='The date in YYYYMMDD format.')

args = parser.parse_args()

# Access arguments
run_number = args.run_number
date = args.date

filename_event_ids = '/fefs/aswg/workspace/vadym.voitsekhovskyi/2024muons/lst_study/real_data/azimuthal_profile_study/{run_number}_muons_complet04_contain099.txt.npy'
event_ids = np.load(filename_event_ids)


file_path = glob.glob('/fefs/aswg/data/real/DL1/{date}/v0.10/tailcut84/dl1_LST-1.Run{run_number}.*.h5')

# Open the .h5 file
for filename in file_path:
    with h5py.File(filename, 'r') as f:
        # Access the dataset
        dataset = f['dl1/event/telescope/image/LST_LSTCam']

        # Read the data
        data = dataset[:]

        for event_record in data:
            if event_record[1] in event_ids:
                filename_image = f'/fefs/aswg/workspace/vadym.voitsekhovskyi/2024muons/lst_study/real_data/azimuthal_profile_study/{run_number}_dl1_image_event{event_record[1]}.txt'
                filename_image_mask = f'/fefs/aswg/workspace/vadym.voitsekhovskyi/2024muons/lst_study/real_data/azimuthal_profile_study/{run_number}_dl1__image_mask_event{event_record[1]}.txt'
                np.save(filename_image, event_record[2])
                np.save(filename_image_mask, event_record[4])

In [ ]:
filename_event_ids = '/Users/vdk/muons2024/data/additional_statistic/event_ids/17769_muons_complet04_contain099.txt.npy'
event_ids = np.load(filename_event_ids)

In [ ]:
image_folder = glob.glob('/Users/vdk/muons2024/data/additional_statistic/event_ids/image_17743/image/*')
mask_folder = glob.glob('/Users/vdk/muons2024/data/additional_statistic/event_ids/image_17743/image_mask/*')


data_dl1_images = []
data_dl1_mask = []

for file in image_folder:
    tmp_result = np.load(file)
    data_dl1_images.append(tmp_result)

for file in mask_folder:
    tmp_result = np.load(file)
    data_dl1_mask.append(tmp_result)


In [ ]:
from ctapipe.image import tailcuts_clean
from ctapipe.visualization import CameraDisplay
import matplotlib.pyplot as plt

camgeom = source.subarray.tel[1].camera.geometry
new_data_mask = []
for image in data_dl1_images:
    mask = tailcuts_clean(camgeom, image, picture_thresh=10, boundary_thresh=5)
    new_data_mask.append(mask)

In [ ]:
image = data_dl1_images[500]

negative_Q = np.sort(image[image <= 0])

hist, bins = np.histogram(negative_Q,range=(-15,0),bins=30)
bins = bins[:-1]

cumulative = np.cumsum(hist)
idx = (np.abs(cumulative - 0.318 * cumulative[-1])).argmin() #Find q closest to standard deviation
dev = np.abs(bins[idx])

tailcuts = [4*dev,2*dev]   # tailcuts are placed at 4*dev of each image.
tailcuts

In [ ]:
new_data_mask_tailcuts = []
for image in data_dl1_images:
    negative_Q = np.sort(image[image <= 0])

    hist, bins = np.histogram(negative_Q,range=(-15,0),bins=30)
    bins = bins[:-1]

    cumulative = np.cumsum(hist)
    idx = (np.abs(cumulative - 0.318 * cumulative[-1])).argmin() #Find q closest to standard deviation
    dev = np.abs(bins[idx])

    tailcuts = [4*dev,2*dev]   # tailcuts are placed at 4*dev of each image.
    
    mask = tailcuts_clean(camgeom, image, picture_thresh=tailcuts[0], boundary_thresh=tailcuts[1])
    new_data_mask_tailcuts.append(mask)

In [ ]:
cur_index = 6

data_dl1_mask = np.array(data_dl1_mask).astype(bool)
fig, ax = plt.subplots(1, 2, figsize=(18, 10))
disp = CameraDisplay(
    camgeom, image=data_dl1_images[cur_index], cmap="grey", ax = ax[0], title="Pixel mask with standard tailcut cleaning [10;5]"
)
disp.highlight_pixels(new_data_mask[cur_index], alpha=0.6, linewidth=2, color="red")
disp = CameraDisplay(
    camgeom, image=data_dl1_images[cur_index], cmap="grey", ax = ax[1], title=f"Pixel mask with modified tailcuts cleaning"
)
disp.highlight_pixels(new_data_mask_tailcuts[cur_index], alpha=0.6, linewidth=2, color="red")

In [ ]:
from ctapipe.image.muon import (
    MuonIntensityFitter,
    MuonRingFitter
)



fitter = MuonRingFitter(fit_method='kundu_chaudhuri')

ring = fitter(x_coord, y_coord, data_dl1_images[2], new_data_mask[2])

for _ in (0, 0):  # just to iterate the fit twice more
    dist = np.sqrt(
        (x_coord - ring.center_fov_lon) ** 2 + (y_coord - ring.center_fov_lat) ** 2
    )
    ring_dist = np.abs(dist - ring.radius)

    new_data_mask[2] *= (ring_dist < ring.radius * 3)
    ring = fitter(x_coord, y_coord, image, new_data_mask[2])   

ring.radius

In [ ]:
data_dl1_mask[1]

In [ ]:
new_data_mask[0]

In [ ]:
max_limit = 50
fig, ax = plt.subplots(1, 2, figsize=(18, 10))
camgeom = source.subarray.tel[1].camera.geometry
#title="CT{}, run {} cog {} fit {} width {}".format(1,event.index.obs_id, round(event.dl1.tel[1].parameters.concentration.cog, 3), round(j.muon.tel[1].efficiency.optical_efficiency, 3), round(j.muon.tel[1].efficiency.width.to_value(),3))
disp = CameraDisplay(
    camgeom, image=data_dl1_images[0], cmap=plt.cm.RdBu_r, ax = ax[0], title="dl1 image sim"
)
#disp.set_limits_percent(20)
disp.set_limits_minmax(0,max_limit)
disp.add_colorbar()
disp = CameraDisplay(
    camgeom, image=data_dl1_images[1], cmap=plt.cm.RdBu_r, ax = ax[1], title="dl1 image data"
)

disp.add_colorbar()
disp.set_limits_minmax(0,max_limit)
#plt.savefig(f"/Users/vdk/TrurImageCheckCOG{i}_COG=0_fitWrong")
plt.show()

In [ ]:
import os
import glob
import re
import numpy as np
from ctapipe.image import tailcuts_clean

image_folder = glob.glob('/Users/vdk/muons2024/data/additional_statistic/event_ids/image_17743/image/*')

file_pattern = '17743_dl1_image_event*.txt.npy'
file_list = glob.glob('/Users/vdk/muons2024/data/additional_statistic/event_ids/image_17743/image/' + file_pattern) 

data_dict = {}

for file_path in file_list:
    filename = os.path.basename(file_path)
    match = re.search(r'event(\d+)\.txt\.npy$', filename)
    if match:
        event_id = int(match.group(1))
        array_content = np.load(file_path)
        data_dict[event_id] = array_content
    else:
        print(f"Filename {filename} does not match the expected pattern.")


camgeom = source.subarray.tel[1].camera.geometry
data_mask = {}
for event_id, array_content in data_dict.items():
    mask = tailcuts_clean(camgeom, array_content, picture_thresh=10, boundary_thresh=5)
    data_mask[event_id] = mask

In [ ]:
disp = CameraDisplay(
    camgeom, image=data_dict[2399474], cmap="gray", title="Pixels with signal"
)
disp.highlight_pixels(data_mask[2399474], alpha=0.8, linewidth=2, color="green")

In [ ]:
test_name = "test_hello_world"

In [ ]:
test_name_2 = test_name.replace('_', '\_').lstrip('test\_')

In [ ]:
test_name_2

In [ ]:
test_name

In [ ]:
test_name.replace('test_', '', 1).replace('_', '\_')

# New processed data

In [ ]:
fits_file = f'/Users/vdk/muons2024/real_data/17744/fits_tables/tables_run{17744}/muons_LST-1.Run17744.0047.fits'

# Read the FITS file into an Astropy Table
dat = Table.read(fits_file, format='fits')

# Ensure 'good_ring' is of boolean type
dat['good_ring'] = dat['good_ring'].astype(bool)

# Identify one-dimensional and multidimensional columns
one_d_cols = [name for name in dat.colnames if len(dat[name].shape) <= 1]
multi_d_cols = [name for name in dat.colnames if len(dat[name].shape) > 1]

# Convert one-dimensional columns to pandas DataFrame
df = dat[one_d_cols].to_pandas()

# Extract multidimensional data separately
images = dat['image'].tolist() if 'image' in multi_d_cols else None
dist_masks = dat['dist_mask'].tolist() if 'dist_mask' in multi_d_cols else None

# Apply your filtering criteria on the DataFrame
df_good_data = df[(df['muon_efficiency'] < 1) & 
                 (df['size_outside'] < 500)]

# Optionally, filter the multidimensional data based on the same criteria
if images is not None:
    images_good = np.array([img for img, cond in zip(images, (df['muon_efficiency'] < 1) &
                                                (df['size_outside'] < 500)) if cond])
if dist_masks is not None:
    dist_masks_good = np.array([mask for mask, cond in zip(dist_masks, (df['muon_efficiency'] < 1) & 
                                                     (df['size_outside'] < 500)) if cond])

In [ ]:
df